#

# Окончательный вариант парсинга

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display

импорт библиотек

In [2]:
import warnings
warnings.filterwarnings('ignore')

отключение ворнингов

In [3]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

задание параметров таблиц

In [4]:
def find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

функция создания столбца с кодом товара

In [5]:
def find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n',''))
    return xxx  

функция создания столбца с парсингом наличия товара

In [6]:
def find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

In [7]:
def find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

In [8]:
def find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

In [9]:
def find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

In [10]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

In [11]:
def parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        while q>0:        
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
            else:
                q = 0
    #data['soup_name'] = data['soup_name'].apply(names_correct)
    data['id'] = data['url'].apply(find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(find_availability)
    data['title'] = data['url'].apply(find_title)
    data['price'] = data['url'].apply(find_price)
    data['old_price'] = data['url'].apply(find_old_price)
    data['url'] = data['url'].apply(find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

# Парсинг цен кодов товара и url

In [12]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

In [13]:
df = parsing_2(all_url)
df

  0%|          | 0/6 [00:00<?, ?it/s]

,url,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-06-12,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-06-12,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-06-12,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-06-12,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-06-12,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN
...,...,...,...,...,...,...,...
9291,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-transparent-led15w-e27-5628-tkl/,Т-000085036,2024-06-12,В наличии,Люстра 4PL SATIPO TRANSPARENT LED15W E27 (5628) (TKL),3279.0,4099.0
9292,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-amber-led15w-e27-5625-tkl/,Т-000085037,2024-06-12,В наличии,Люстра 4PL SATIPO AMBER LED15W E27 (5625) (TKL),3311.0,4139.0
9293,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-3pl-universo-black-led15w-e27-4483-tkl/,Т-000085021,2024-06-12,В наличии,Люстра 3PL UNIVERSO BLACK LED15W E27 (4483) (TKL),3359.0,4199.0
9294,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-6pl-fairy-black-6w-g9-4856-tkl/,Т-000085033,2024-06-12,В наличии,Люстра 6PL FAIRY BLACK 6W G9 (4856) (TKL),3471.0,4339.0


In [14]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-06-12,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-06-12,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-06-12,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-06-12,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-06-12,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 9296 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:573
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       6846
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       73.6
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9296 entries, 0 to 9295
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           9296 non-null   object        
 1   id            9296 non-null   object        
 2   date          9296 non-null   datetime64[ns]
 3   availability  9296 non-null   object        
 4   title         9296 non-null   object        
 5   price         9296 non-null   float64       
 6   old_price     2450 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 508.5+ KB


None

In [15]:
df = df.drop_duplicates()

In [16]:
def count_sity(x):
    try:
        reqx = requests.get(x)
        soup = bs(reqx.text, 'html.parser')    
        element = soup.findAll('div', class_='ty-warehouses-store__availability')
        elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
        name = soup.findAll('span', class_='ty-breadcrumbs__current')
        data = []
        try:
            data.append(element[0].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[1].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[2].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[3].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[4].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[5].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[6].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[7].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[8].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[9].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[10].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[11].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[12].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:
            data.append(element[13].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
            .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        except:
            data.append(np.NaN)
        try:        
            if name[0].text.find('/')<0:
                ss = elem_2[0].text
            else:
                ss = elem_2[0].text[:elem_2[0].text.rfind('/')] + ' ' + elem_2[0].text[elem_2[0].text.rfind('/')+1:]           

            if len(ss.split('/')) == 3:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 4:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[2])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 5:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[3]) 
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 6:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-3])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 7:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-4])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 8:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-5])  
                data.append(len(ss.split('/')))
            elif len(ss.split('/')) == 9:
                data.append(ss.split("/")[1])
                data.append(ss.split("/")[2])
                data.append(ss.split("/")[-6]) 
                data.append(len(ss.split('/')))
            return data
        except:
            data.append(np.NaN)
            data.append(np.NaN)
            data.append(np.NaN)
            data.append(np.NaN)
            return data
    except:
        pass

In [ ]:
tqdm.pandas()
df['count_sity']= df['url'].progress_apply(count_sity)

  0%|          | 0/8723 [00:00<?, ?it/s]

In [ ]:
df['тир_юности_14а'] = df['count_sity'].str[0]
df['тир_25_октября_69'] = df['count_sity'].str[1]
df['бенд_спортивная_5'] = df['count_sity'].str[2]
df['бенд_советская_5'] = df['count_sity'].str[3]
df['бенд_советская_8'] = df['count_sity'].str[4]
df['слободзея'] = df['count_sity'].str[5]
df['днестровск'] = df['count_sity'].str[6]
df['первомайск'] = df['count_sity'].str[7]
df['григ_кмаркса_181а'] = df['count_sity'].str[8]
df['григ_кмаркса_172'] = df['count_sity'].str[9]
df['дубоссары'] = df['count_sity'].str[10]
df['рыбн_вальченко_43'] = df['count_sity'].str[11]
df['рыбн_кирова_135'] = df['count_sity'].str[12]
df['каменка'] = df['count_sity'].str[13]
df['_category'] = df['count_sity'].str[14]
df['_subcategory'] = df['count_sity'].str[15]
df['_group'] = df['count_sity'].str[16]
df['len'] = df['count_sity'].str[17]

In [ ]:
df.drop(['count_sity'], axis= 1 , inplace= True )

In [ ]:
preprocessing(df)

In [ ]:
df['error_count'] = 0
df.loc[df['каменка'].isna(), 'error_count'] = 1
df['error_group'] = 0
df.loc[df['len'].isna(), 'error_group'] = 1

In [ ]:
df.loc[df['error_count']==1 , ['тир_юности_14а', 'ц_тир_25_октября_69', 'бенд_спортивная_5',
       'бенд_советская_5', 'бенд_советская_8', 'слободзея', 'днестровск',
       'первомайск', 'григ_кмаркса_181а', 'григ_кмаркса_172', 'дубоссары',
       'рыбн_вальченко_43', 'рыбн_кирова_135', 'каменка']] = 0
df.loc[df['error_group']==1 , ['_category',
       '_subcategory', '_group']] = 'No_group'
df.loc[df['error_group']==1 , ['len']] = 0

In [ ]:
df['тир_юности_14а'] = df['тир_юности_14а'].astype('int')
df['тир_25_октября_69'] = df['тир_25_октября_69'].astype('int')
df['тирасполь'] = df['тир_юности_14а'] + df['тир_25_октября_69']
df['бенд_спортивная_5'] = df['бенд_спортивная_5'].astype('int')
df['бенд_советская_5'] = df['бенд_советская_5'].astype('int')
df['бенд_советская_8'] = df['бенд_советская_8'].astype('int')
df['бендеры'] = df['бенд_спортивная_5'] + df['бенд_советская_5'] + df['бенд_советская_8']
df['слободзея'] = df['слободзея'].astype('int')
df['днестровск'] = df['днестровск'].astype('int')
df['первомайск'] = df['первомайск'].astype('int')
df['григ_кмаркса_181а'] = df['григ_кмаркса_181а'].astype('int')
df['григ_кмаркса_172'] = df['григ_кмаркса_172'].astype('int')
df['григориополь'] = df['григ_кмаркса_181а'] + df['григ_кмаркса_172']
df['дубоссары'] = df['дубоссары'].astype('int')
df['рыбн_вальченко_43'] = df['рыбн_вальченко_43'].astype('int')
df['рыбн_кирова_135'] = df['рыбн_кирова_135'].astype('int')
df['рыбница'] = df['рыбн_вальченко_43'] + df['рыбн_кирова_135']
df['каменка'] = df['каменка'].astype('int')

In [ ]:
df.columns = [['url', 'id', '_category', '_subcategory', '_group', 'title',
               'date',   'price', 'old_price', 'availability', 'тирасполь',
               'тир_юности_14а', 'тир_25_октября_69', 'бендеры', 'бенд_спортивная_5',
               'бенд_советская_5', 'бенд_советская_8', 'рыбница', 'рыбн_вальченко_43', 
               'рыбн_кирова_135', 'дубоссары', 'днестровск', 'слободзея', 'первомайск',
               'каменка', 'григориополь','григ_кмаркса_181а', 'григ_кмаркса_172',
               'len', 'error_count', 'error_group', ]]

In [ ]:
dd = ['Батарейки и зарядные устройства', 'Крепления для тв',
       'Аудиосистемы', 'Смартфоны и аксессуары', 'Мультимедиа',
       'Приставки для ТВ', 'Гаджеты', 'Телефоны dect и проводные',
       'Apple', 'Видеотехника', 'Товар без акции', 'Умный дом с Яндекс',
       'Телевизоры', 'Пылесосы и аксессуары', 'Техника karcher',
       'Аксессуары для мелкой бытовой техники',
       'Товары для ухода за одеждой', 'Минимойки',
       'Приготовление напитков', 'Мелкая техника для кухни',
       'Техника для красоты', 'Стиральные и сушильные машины',
       'Крупная техника для кухни', 'Техника для здоровья',
       'Климатическая техника', 'Установка бытовой техники',
       'Приготовление пищи', 'Техника для дома', 'Накопители',
       'Сетевое оборудование', 'Аксессуары', 'Расходные материалы',
       'Периферийные устройства', 'Оргтехника', 'Комплектующие',
       'Крепления', 'Электропитание', 'Игровые приставки',
       'Товар по акции', 'Мониторы', 'Проекторы и экраны',
       'Готовые решения', 'Мебель для офиса', 'Электроинструменты',
       'Сервировка стола', 'Кухонные принадлежности', 'Освещение',
       'Посуда для духовок и микроволновых печей', 'Бытовая химия']

In [ ]:
for i in dd:
    t = df[df['_subcategory']==i]
    print(i)
    print(t._group.unique())
    print('-'*100)